In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

In [3]:
def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format( \
                subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

In [4]:
subreddit = 'Coffee'
end_at = math.ceil(datetime.utcnow().timestamp())
start_at = math.floor((datetime.utcnow() - \              
                       timedelta(days=365)).timestamp())
posts = pull_posts_for(subreddit, start_at, end_at)

NameError: name 'make_request' is not defined

In [ ]:
class RedditSpider(scrapy.Spider):
    name = "reddit"
    allowed_domains = ["reddit.com"]
    start_urls = [
        "https://www.reddit.com/r/Coffee"
    ]
    
    def parse(self, response): 
        hxs = Selector(response) 
        for item in hxs.xpath("//p[@class='_1qeIAgB0cPwnLhDF9XSiJM']/text()").extract():
            items.append(item)
        return items

pd.DataFrame(items)

In [ ]:
# unwanted

import pandas as pd

import requests

def get_reddit(subreddit=None, listing=None, limit=None, timeframe=None):
    
    try:
        
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'},
                              verify=False)
        
    except:
        
        print('An Error Occured')
        
    return request.json()

r = get_reddit(subreddit='tea', listing='hot', limit=1000, timeframe='all')

def get_post_titles(r):

    posts = []
    
    for post in r['data']['children']:
        x = post['data']['title']
        posts.append(x)
        
    return posts

posts = get_post_titles(r)

def get_results(r):

    myDict = {}
    
    for post in r['data']['children']:
        
        myDict[post['data']['title']] = {'url':post['data']['url'],
                                         'post_url':'https://www.reddit.com'+post['data']['permalink'],
                                         'score':post['data']['score'],
                                         'comments':post['data']['num_comments']}
        
    df = pd.DataFrame.from_dict(myDict, orient='index')
    
    return df

df = get_results(r)

def get_reddit_post(url):
    
    try:
        
        url = url + '.json'
        
        request = requests.get(url, headers = {'User-agent': 'yourbot'},
                              verify=False)
        
    except:
        
        print('An Error Occured')
        
    content = request.json()
    
    for d in content:
        
        for dictionary in d['data']['children']:
            
            c = dictionary['data']['selftext']
            
            break
            
        break
        
    return c

df['post_content'] = df.post_url.map(get_reddit_post)

In [ ]:
# to run the tests again by adding coffee and tea into stopwords 

# include the common words found on the threads
english_stopwords = stopwords.words('english')
english_stopwords2 = english_stopwords.extend(['hi', 'all', 'everyone', 'thank you', 'hello', 'mod', 'edit', 'etc', 'im', 'me', 'ive', 'thank', 'coffee', 'tea'])

def filtered_text_2(words):
    """Remove common english stop words from an array, and those which are numbers"""
    filtered_words_2 = [word for word in words if (word not in english_stopwords2) and (not word.isdigit())]
    return filtered_words_2

In [ ]:
# filter away the coffee and tea 
merged_df['filtered_tokens_2'] = merged_df['tokens'].apply(filtered_text)

In [ ]:
# then lemmatize it
merged_df['lem_tokens_2'] = merged_df['filtered_tokens_2'].apply(lemmatize_text)

In [ ]:
text = merged_df["lem_tokens_2"].map(' '.join)
X = text
y = merged_df['category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
cvec2 = CountVectorizer(min_df=.01)

In [ ]:
cvec2.get_feature_names()

In [ ]:
X_train_df = pd.DataFrame(cvec2.fit_transform(X_train).todense(), 
                          columns=cvec2.get_feature_names())

# plot top occuring words
X_train_df.sum().sort_values(ascending=False).head(40).plot(kind='barh');

In [ ]:
pipe2 = Pipeline([
    ('cvec', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [ ]:
pipe_params = {
    'cvec__max_features': [2_000, 3_000, 4_000, 5_000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}

In [ ]:
gs2 = GridSearchCV(pipe2, 
                  param_grid=pipe_params, 
                  cv=5)

In [ ]:
gs2.fit(X_train, y_train)

In [ ]:
gs2.score(X_train, y_train)

In [ ]:
gs2.score(X_test, y_test)

In [ ]:
# Get predictions
preds = gs2.predict(X_test)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

In [ ]:
# View confusion matrix
plot_confusion_matrix(gs, X_test, y_test, cmap='Blues', values_format='d');

In [ ]:
spec = tn / (tn + fp)
print('Specificity for count vectorizer with Naive Bayes:', spec)

# this performs worse than the previous one